In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM, Flatten, Conv1D, LocallyConnected1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from math import sqrt
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from matplotlib import pyplot
import keras
from sklearn.preprocessing import OneHotEncoder
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import numpy as np
import argparse
import os
from keras.callbacks import CSVLogger
import keras.backend as K


Using TensorFlow backend.


## DATA PREP

In [2]:
import data_preparation
#Convert letters to integers
integer_encoded, params = data_preparation.process_data('input.txt')

15072434
{'C': 0, 'G': 1, 'T': 2, 'A': 3}
{0: 'C', 1: 'G', 2: 'T', 3: 'A'}
[[1]
 [0]
 [0]
 [2]
 [3]
 [3]
 [1]
 [0]
 [0]
 [2]]
GCCTAAGCCT


In [3]:
bs=128
time_steps=64
num_epochs=20

In [4]:
#Generate traning data and its lable 
X,Y_raw, Y = data_preparation.generate_single_output_data(integer_encoded[: bs * 90 + 95 ],bs, time_steps)


(11615, 1)
(11615,)
11520
(11520, 65)


C:\Users\quoca\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Trainning

In [11]:
def custom_loss(y_true, y_pred):
        return 1/np.log(2) * K.categorical_crossentropy(y_true, y_pred)        

In [45]:
alphabet_size = Y.shape[1]
DZ_model = Sequential()
DZ_model.add(Embedding(alphabet_size, 32, batch_input_shape=(bs, time_steps)))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(LSTM(32, stateful=False, return_sequences=True))
DZ_model.add(Flatten())
DZ_model.add(Dense(64, activation='relu'))
DZ_model.add(Dense(alphabet_size, activation='softmax'))
optim = keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0, amsgrad=False)
DZ_model.compile(loss=custom_loss, optimizer=optim)
early_stopping = EarlyStopping(monitor='loss', mode='min', min_delta=0.005, patience=3, verbose=1)
callbacks_list = [early_stopping]
DZ_model.fit(X, Y, epochs=num_epochs, batch_size=bs, verbose=1, shuffle=True, callbacks=callbacks_list)

Epoch 1/20
384/384 [==============================] - 2s 5ms/step - loss: 1.9931
Epoch 2/20
384/384 [==============================] - 0s 595us/step - loss: 1.9572
Epoch 3/20
384/384 [==============================] - 0s 600us/step - loss: 1.9248
Epoch 4/20
384/384 [==============================] - 0s 604us/step - loss: 1.9080
Epoch 5/20
384/384 [==============================] - 0s 599us/step - loss: 1.8920
Epoch 6/20
384/384 [==============================] - 0s 603us/step - loss: 1.8743
Epoch 7/20
384/384 [==============================] - 0s 627us/step - loss: 1.8528
Epoch 8/20
384/384 [==============================] - 0s 631us/step - loss: 1.8149
Epoch 9/20
384/384 [==============================] - 0s 642us/step - loss: 1.7636
Epoch 10/20
384/384 [==============================] - 0s 613us/step - loss: 1.6873
Epoch 11/20
384/384 [==============================] - 0s 591us/step - loss: 1.5822
Epoch 12/20
384/384 [==============================] - 0s 608us/step - loss: 1.4403
Epo

## Compression

In [5]:
DZ_model = load_model('my_model.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen   #bit precision 
        self.max_range = 1 << self.bit_prec  #max range based on bit precision 2^bit_prec
        self.mask = self.max_range - 1  #max range index starting at 0
        self.renorm= self.max_range >> 1  #renormalization threshold
        self.second_mask = self.max_range >> 1
        self.low = 0  #initial low
        self.high = self.mask  #initial high
        self.s = 0  

    def update(self, sym, c):
        low = self.low   
        high = self.high
        range = high - low + 1
        print(sym)
        print(sym.shape )
        sym = int(sym)
        total = (c[-1])  #cumulative propabilities
        symlow = (c[sym])  
        symhigh = (c[sym+1] )
        

        newlow = low + symlow*range // total  #low in arithmetic integer
        newhigh = low + symhigh*range // total -1 #high in arithemtic integer
        self.low = int(newlow)
        self.high = int(newhigh)
        range = self.high - self.low 
#         print("sym:", sym)
#         print("symlow:", self.low)
#         print("symhigh:", self.high)
#         print("range:", range)
        
        #renormalization
        while((self.low ^ self.high) & self.renorm) == 0:
            self.low = (self.low << 1) 
            range = range << 1 | 1
            self.high = self.low + range
            self.s = self.s + 1          
#         print("renorm low:",self.low)
#         print("renorm range:",range)
#         print("renorm high:", self.high)
   
    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        
    def finish(self):
        self.output.write(1)


# In[25]:


class ArithmeticDecoder(ArithmeticEncoder):
    def __init__(self,statesize,bitin):
        self.input = bitin
        self.bitstream = ([int(d) for d in str(self.input)])
#         print(self.bitstream)
        self.code = 0
        self.max_range = 1 << statesize
        self.renorm= self.max_range >> 1
        self.mask = self.max_range - 1
        
        self.stream = self.input[0:statesize]
        self.low = int(self.stream,2)
        self.t = statesize
        self.thresh = []
        self.thresh = [round(c[i]*self.mask) for i in range(len(c))]
        
    def decode(self):
        for i in range(len(self.thresh)-1):
            if((self.low < self.thresh[i+1]) & (self.low >= self.thresh[i])):
                sym = i
#                 print("encoded value:",self.low)
#                 print("threshold:", self.thresh)
#                 print("decoded value:", sym)
                rangenew = int(self.thresh[i+1] - self.thresh[i])
#                 print("range:", rangenew)
#                 print("low:", self.thresh[i])
                while (rangenew < self.renorm):
                    rangenew = rangenew << 1 
                    self.low = self.low << 1
                    self.thresh[i] = int(self.thresh[i]) << 1
#                     print("renorm val;",self.low)
#                     print("renorm range;",rangenew)
                    self.thresh[i+1] = rangenew + self.thresh[i]
#                     print("renorm low:" ,self.thresh[i])
                self.thresh = [round(c[j]*(self.thresh[i+1]-self.thresh[i]))+self.thresh[i] for j in range(len(c))]
                break
        return sym


In [21]:
alphabet_size = 4
def cumul_d(prob):
    return np.cumsum(prob, axis = 1)  


In [25]:
bitprecision = 64
l = int(X.shape[0] / bs) * bs 
print(l)
print((X.shape[0])-time_steps)

#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
c = np.zeros(alphabet_size+1,  dtype = np.uint64)
c[1:] = np.cumsum(prob*10000000 + 1)        
print(c)
bitstream = []
enc = ArithmeticEncoder(bitprecision)

num_iters = int((len(X)+time_steps)/bs)
ind = np.array(range(bs))*num_iters
prob = DZ_model.predict(X[ind,:], batch_size=bs)
print(num_iters)

#Encoding
for i in (range(bs)):
    for j in range(min(time_steps, num_iters)):
        print('i =' + str(i))
        print('j =' + str(j))
        if(i == 127 and j == 63):
            print(X[ind[i], j])
            print(c.tolist())
        new = enc.write( X[ind[i], j], c.tolist())
        
low_final = new[0] #fina low value 
s_final = new[1]  #count of renormalizations
ind = ind + 1
#convert low to bitstream, ensure proper number of bits 
bitstream_n = format(int(low_final), 'b')  
cumul = np.zeros((bs, alphabet_size), dtype = np.uint64)        
for k in (range(num_iters - time_steps)):
    prob = DZ_model.predict(X[ind,:], batch_size=bs)
    cumul[:,0:] = np.cumsum(prob*10000000 + 1, axis = 1)
    new = enc.write( X[ind,:], cumul[i,:].tolist())
    for i in range(bs):
        print(cumul[i,:].tolist())
        enc.write(Y_raw[ind[i]], cumul[i,:].tolist())
    ind = ind + 1
    
    
    
    
    
low_final = new[0] #fina low value 
s_final = new[1]  #count of renormalizations
ind = ind + 1
#convert low to bitstream, ensure proper number of bits 
bitstream_n = format(int(low_final), 'b')  
while len(bitstream_n) < (s_final+bitprecision):
    bitstream = '0' + bitstream_n
print("encoded bit length:", len(bitstream))
   
    
# else:
#     prob = np.ones(alphabet_size)/alphabet_size
#     cumul = np.zeros(alphabet_size+1, dtype = float)
#     cumul[1:] = np.cumsum(prob)         
#     print(cumul.tolist())
#     enc = ArithmeticEncoder(bitprecision)
#     for j in range(time_steps):
#         new = enc.write( X[0,j], cumul.tolist())
#     for i in (range(len(X))):
#         prob = DZ_model.predict(X[i,:].reshape(1,-1), batch_size=1)
#         cumul[1:] = np.cumsum(prob)
#         new = enc.write( Y_raw[i][0], cumul)
#     #convert low to bitstream, ensure proper number of bits 
#     bitstream = format(int(low_final), 'b')  
#     while len(bitstream) < (s_final+bitprecision):
#         bitstream = '0' + bitstream
#     print("encoded bit length:", len(bitstream))   

 #START DECODER
print(bitstream)
dec = ArithmeticDecoder(bitprecision, bitstream)

symdec = []
for symbols in range(len(sym)):
    symdec.append(dec.decode())
print("decoded num stream:", symdec)




11520
11456
[       0  2500001  5000002  7500003 10000004]
90
i =0
j =0
1
()
i =0
j =1
0
()
i =0
j =2
0
()
i =0
j =3
2
()
i =0
j =4
3
()
i =0
j =5
3
()
i =0
j =6
1
()
i =0
j =7
0
()
i =0
j =8
0
()
i =0
j =9
2
()
i =0
j =10
3
()
i =0
j =11
3
()
i =0
j =12
1
()
i =0
j =13
0
()
i =0
j =14
0
()
i =0
j =15
2
()
i =0
j =16
3
()
i =0
j =17
3
()
i =0
j =18
1
()
i =0
j =19
0
()
i =0
j =20
0
()
i =0
j =21
2
()
i =0
j =22
3
()
i =0
j =23
3
()
i =0
j =24
1
()
i =0
j =25
0
()
i =0
j =26
0
()
i =0
j =27
2
()
i =0
j =28
3
()
i =0
j =29
3
()
i =0
j =30
1
()
i =0
j =31
0
()
i =0
j =32
0
()
i =0
j =33
2
()
i =0
j =34
3
()
i =0
j =35
3
()
i =0
j =36
1
()
i =0
j =37
0
()
i =0
j =38
0
()
i =0
j =39
2
()
i =0
j =40
3
()
i =0
j =41
3
()
i =0
j =42
1
()
i =0
j =43
0
()
i =0
j =44
0
()
i =0
j =45
2
()
i =0
j =46
3
()
i =0
j =47
3
()
i =0
j =48
1
()
i =0
j =49
0
()
i =0
j =50
0
()
i =0
j =51
2
()
i =0
j =52
3
()
i =0
j =53
3
()
i =0
j =54
1
()
i =0
j =55
0
()
i =0
j =56
0
()
i =0
j =57
2
()
i =0
j =58
3
()
i =0

i =13
j =0
2
()
i =13
j =1
2
()
i =13
j =2
2
()
i =13
j =3
2
()
i =13
j =4
1
()
i =13
j =5
1
()
i =13
j =6
3
()
i =13
j =7
3
()
i =13
j =8
3
()
i =13
j =9
2
()
i =13
j =10
2
()
i =13
j =11
3
()
i =13
j =12
1
()
i =13
j =13
2
()
i =13
j =14
2
()
i =13
j =15
2
()
i =13
j =16
3
()
i =13
j =17
3
()
i =13
j =18
3
()
i =13
j =19
3
()
i =13
j =20
3
()
i =13
j =21
2
()
i =13
j =22
0
()
i =13
j =23
2
()
i =13
j =24
0
()
i =13
j =25
3
()
i =13
j =26
0
()
i =13
j =27
3
()
i =13
j =28
2
()
i =13
j =29
2
()
i =13
j =30
2
()
i =13
j =31
2
()
i =13
j =32
2
()
i =13
j =33
2
()
i =13
j =34
2
()
i =13
j =35
2
()
i =13
j =36
3
()
i =13
j =37
1
()
i =13
j =38
3
()
i =13
j =39
3
()
i =13
j =40
3
()
i =13
j =41
3
()
i =13
j =42
3
()
i =13
j =43
2
()
i =13
j =44
2
()
i =13
j =45
3
()
i =13
j =46
2
()
i =13
j =47
2
()
i =13
j =48
2
()
i =13
j =49
2
()
i =13
j =50
2
()
i =13
j =51
3
()
i =13
j =52
3
()
i =13
j =53
1
()
i =13
j =54
3
()
i =13
j =55
3
()
i =13
j =56
2
()
i =13
j =57
2
()
i =13
j =58
2
()
i =13
j

()
i =20
j =52
1
()
i =20
j =53
1
()
i =20
j =54
3
()
i =20
j =55
0
()
i =20
j =56
1
()
i =20
j =57
3
()
i =20
j =58
0
()
i =20
j =59
3
()
i =20
j =60
2
()
i =20
j =61
2
()
i =20
j =62
1
()
i =20
j =63
2
()
i =21
j =0
3
()
i =21
j =1
1
()
i =21
j =2
3
()
i =21
j =3
2
()
i =21
j =4
1
()
i =21
j =5
3
()
i =21
j =6
0
()
i =21
j =7
1
()
i =21
j =8
1
()
i =21
j =9
1
()
i =21
j =10
3
()
i =21
j =11
0
()
i =21
j =12
0
()
i =21
j =13
3
()
i =21
j =14
3
()
i =21
j =15
2
()
i =21
j =16
2
()
i =21
j =17
0
()
i =21
j =18
2
()
i =21
j =19
2
()
i =21
j =20
2
()
i =21
j =21
0
()
i =21
j =22
2
()
i =21
j =23
1
()
i =21
j =24
2
()
i =21
j =25
0
()
i =21
j =26
3
()
i =21
j =27
1
()
i =21
j =28
1
()
i =21
j =29
2
()
i =21
j =30
2
()
i =21
j =31
2
()
i =21
j =32
2
()
i =21
j =33
3
()
i =21
j =34
0
()
i =21
j =35
3
()
i =21
j =36
3
()
i =21
j =37
0
()
i =21
j =38
0
()
i =21
j =39
1
()
i =21
j =40
0
()
i =21
j =41
0
()
i =21
j =42
0
()
i =21
j =43
3
()
i =21
j =44
1
()
i =21
j =45
2
()
i =21
j =46
1
()
i =2

()
i =28
j =40
0
()
i =28
j =41
2
()
i =28
j =42
2
()
i =28
j =43
3
()
i =28
j =44
2
()
i =28
j =45
2
()
i =28
j =46
2
()
i =28
j =47
3
()
i =28
j =48
0
()
i =28
j =49
1
()
i =28
j =50
0
()
i =28
j =51
0
()
i =28
j =52
3
()
i =28
j =53
3
()
i =28
j =54
1
()
i =28
j =55
3
()
i =28
j =56
3
()
i =28
j =57
3
()
i =28
j =58
2
()
i =28
j =59
3
()
i =28
j =60
3
()
i =28
j =61
3
()
i =28
j =62
2
()
i =28
j =63
2
()
i =29
j =0
3
()
i =29
j =1
3
()
i =29
j =2
3
()
i =29
j =3
3
()
i =29
j =4
3
()
i =29
j =5
3
()
i =29
j =6
3
()
i =29
j =7
2
()
i =29
j =8
3
()
i =29
j =9
3
()
i =29
j =10
2
()
i =29
j =11
2
()
i =29
j =12
3
()
i =29
j =13
3
()
i =29
j =14
3
()
i =29
j =15
3
()
i =29
j =16
3
()
i =29
j =17
3
()
i =29
j =18
3
()
i =29
j =19
0
()
i =29
j =20
3
()
i =29
j =21
0
()
i =29
j =22
3
()
i =29
j =23
2
()
i =29
j =24
2
()
i =29
j =25
2
()
i =29
j =26
2
()
i =29
j =27
2
()
i =29
j =28
2
()
i =29
j =29
1
()
i =29
j =30
1
()
i =29
j =31
3
()
i =29
j =32
3
()
i =29
j =33
3
()
i =29
j =34
3
()
i =2

1
()
i =36
j =28
3
()
i =36
j =29
2
()
i =36
j =30
3
()
i =36
j =31
2
()
i =36
j =32
2
()
i =36
j =33
2
()
i =36
j =34
3
()
i =36
j =35
3
()
i =36
j =36
2
()
i =36
j =37
1
()
i =36
j =38
1
()
i =36
j =39
3
()
i =36
j =40
3
()
i =36
j =41
2
()
i =36
j =42
2
()
i =36
j =43
1
()
i =36
j =44
2
()
i =36
j =45
2
()
i =36
j =46
2
()
i =36
j =47
2
()
i =36
j =48
2
()
i =36
j =49
1
()
i =36
j =50
2
()
i =36
j =51
2
()
i =36
j =52
2
()
i =36
j =53
2
()
i =36
j =54
2
()
i =36
j =55
3
()
i =36
j =56
2
()
i =36
j =57
2
()
i =36
j =58
3
()
i =36
j =59
3
()
i =36
j =60
2
()
i =36
j =61
3
()
i =36
j =62
3
()
i =36
j =63
3
()
i =37
j =0
3
()
i =37
j =1
3
()
i =37
j =2
2
()
i =37
j =3
2
()
i =37
j =4
2
()
i =37
j =5
2
()
i =37
j =6
0
()
i =37
j =7
3
()
i =37
j =8
3
()
i =37
j =9
2
()
i =37
j =10
3
()
i =37
j =11
3
()
i =37
j =12
3
()
i =37
j =13
3
()
i =37
j =14
3
()
i =37
j =15
3
()
i =37
j =16
0
()
i =37
j =17
3
()
i =37
j =18
0
()
i =37
j =19
0
()
i =37
j =20
3
()
i =37
j =21
3
()
i =37
j =22
3
()
i 

...

()
i =93
j =4
1
()
i =93
j =5
3
()
i =93
j =6
1
()
i =93
j =7
0
()
i =93
j =8
3
()
i =93
j =9
0
()
i =93
j =10
3
()
i =93
j =11
0
()
i =93
j =12
0
()
i =93
j =13
1
()
i =93
j =14
3
()
i =93
j =15
2
()
i =93
j =16
2
()
i =93
j =17
3
()
i =93
j =18
3
()
i =93
j =19
1
()
i =93
j =20
2
()
i =93
j =21
2
()
i =93
j =22
2
()
i =93
j =23
0
()
i =93
j =24
2
()
i =93
j =25
0
()
i =93
j =26
0
()
i =93
j =27
3
()
i =93
j =28
1
()
i =93
j =29
2
()
i =93
j =30
1
()
i =93
j =31
1
()
i =93
j =32
2
()
i =93
j =33
3
()
i =93
j =34
1
()
i =93
j =35
3
()
i =93
j =36
1
()
i =93
j =37
2
()
i =93
j =38
2
()
i =93
j =39
2
()
i =93
j =40
2
()
i =93
j =41
2
()
i =93
j =42
2
()
i =93
j =43
0
()
i =93
j =44
2
()
i =93
j =45
3
()
i =93
j =46
1
()
i =93
j =47
1
()
i =93
j =48
0
()
i =93
j =49
0
()
i =93
j =50
3
()
i =93
j =51
2
()
i =93
j =52
0
()
i =93
j =53
3
()
i =93
j =54
2
()
i =93
j =55
2
()
i =93
j =56
3
()
i =93
j =57
2
()
i =93
j =58
1
()
i =93
j =59
0
()
i =93
j =60
2
()
i =93
j =61
3
()
i =93
j =62
2
()


1
()
i =102
j =53
1
()
i =102
j =54
1
()
i =102
j =55
1
()
i =102
j =56
1
()
i =102
j =57
3
()
i =102
j =58
1
()
i =102
j =59
0
()
i =102
j =60
3
()
i =102
j =61
2
()
i =102
j =62
2
()
i =102
j =63
2
()
i =103
j =0
3
()
i =103
j =1
1
()
i =103
j =2
2
()
i =103
j =3
0
()
i =103
j =4
3
()
i =103
j =5
2
()
i =103
j =6
0
()
i =103
j =7
3
()
i =103
j =8
2
()
i =103
j =9
2
()
i =103
j =10
2
()
i =103
j =11
2
()
i =103
j =12
2
()
i =103
j =13
3
()
i =103
j =14
1
()
i =103
j =15
0
()
i =103
j =16
3
()
i =103
j =17
1
()
i =103
j =18
2
()
i =103
j =19
2
()
i =103
j =20
2
()
i =103
j =21
0
()
i =103
j =22
2
()
i =103
j =23
1
()
i =103
j =24
2
()
i =103
j =25
1
()
i =103
j =26
2
()
i =103
j =27
3
()
i =103
j =28
3
()
i =103
j =29
3
()
i =103
j =30
3
()
i =103
j =31
2
()
i =103
j =32
2
()
i =103
j =33
0
()
i =103
j =34
1
()
i =103
j =35
0
()
i =103
j =36
1
()
i =103
j =37
2
()
i =103
j =38
3
()
i =103
j =39
1
()
i =103
j =40
3
()
i =103
j =41
2
()
i =103
j =42
0
()
i =103
j =43
3
()
i =103
j =44
0


2
()
i =112
j =38
1
()
i =112
j =39
2
()
i =112
j =40
1
()
i =112
j =41
3
()
i =112
j =42
3
()
i =112
j =43
2
()
i =112
j =44
2
()
i =112
j =45
0
()
i =112
j =46
2
()
i =112
j =47
3
()
i =112
j =48
1
()
i =112
j =49
3
()
i =112
j =50
3
()
i =112
j =51
1
()
i =112
j =52
3
()
i =112
j =53
2
()
i =112
j =54
1
()
i =112
j =55
3
()
i =112
j =56
2
()
i =112
j =57
1
()
i =112
j =58
3
()
i =112
j =59
3
()
i =112
j =60
1
()
i =112
j =61
3
()
i =112
j =62
1
()
i =112
j =63
0
()
i =113
j =0
0
()
i =113
j =1
0
()
i =113
j =2
1
()
i =113
j =3
3
()
i =113
j =4
1
()
i =113
j =5
0
()
i =113
j =6
3
()
i =113
j =7
3
()
i =113
j =8
2
()
i =113
j =9
0
()
i =113
j =10
2
()
i =113
j =11
1
()
i =113
j =12
1
()
i =113
j =13
3
()
i =113
j =14
1
()
i =113
j =15
3
()
i =113
j =16
0
()
i =113
j =17
2
()
i =113
j =18
2
()
i =113
j =19
2
()
i =113
j =20
2
()
i =113
j =21
1
()
i =113
j =22
2
()
i =113
j =23
1
()
i =113
j =24
3
()
i =113
j =25
0
()
i =113
j =26
1
()
i =113
j =27
3
()
i =113
j =28
3
()
i =113
j =29
3


j =22
3
()
i =122
j =23
3
()
i =122
j =24
3
()
i =122
j =25
3
()
i =122
j =26
2
()
i =122
j =27
3
()
i =122
j =28
2
()
i =122
j =29
2
()
i =122
j =30
2
()
i =122
j =31
2
()
i =122
j =32
0
()
i =122
j =33
0
()
i =122
j =34
0
()
i =122
j =35
1
()
i =122
j =36
2
()
i =122
j =37
3
()
i =122
j =38
3
()
i =122
j =39
2
()
i =122
j =40
3
()
i =122
j =41
3
()
i =122
j =42
0
()
i =122
j =43
2
()
i =122
j =44
0
()
i =122
j =45
2
()
i =122
j =46
3
()
i =122
j =47
0
()
i =122
j =48
2
()
i =122
j =49
0
()
i =122
j =50
0
()
i =122
j =51
1
()
i =122
j =52
3
()
i =122
j =53
0
()
i =122
j =54
3
()
i =122
j =55
3
()
i =122
j =56
3
()
i =122
j =57
0
()
i =122
j =58
3
()
i =122
j =59
2
()
i =122
j =60
2
()
i =122
j =61
3
()
i =122
j =62
0
()
i =122
j =63
1
()
i =123
j =0
1
()
i =123
j =1
2
()
i =123
j =2
1
()
i =123
j =3
3
()
i =123
j =4
1
()
i =123
j =5
3
()
i =123
j =6
0
()
i =123
j =7
2
()
i =123
j =8
0
()
i =123
j =9
2
()
i =123
j =10
2
()
i =123
j =11
1
()
i =123
j =12
2
()
i =123
j =13
3
()
i =123
j 

TypeError: only size-1 arrays can be converted to Python scalars

In [24]:
cumul.shape

(128, 4)

In [38]:
cumul[:,0:]

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0,

In [16]:
np.asscalar(cumul[-1])


ValueError: can only convert an array of size 1 to a Python scalar

In [38]:
X[i,:].reshape(1,-1).shape

(1, 64)

In [39]:
if (l > (len(X.shape[0])-timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
            enc[i].write(cumul[i,:], y_original[ind[i]])
    ind = ind + 1
else:

SyntaxError: unexpected EOF while parsing (<ipython-input-39-e07fab0c4370>, line 7)

In [8]:
DZ_model = load_model('my_model.h5', compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
#Create Uniform distribution to feed in to the model
alphabet_size = 4
prob = np.ones(alphabet_size)/alphabet_size
#make an array of cumulative probabilities
c = []
for i in range(len(prob)+1):
    c.append(sum(prob[0:i]))

cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
for j in (range(num_iters - timesteps)):
    prob = model.predict(X[ind,:], batch_size=bs)
    cumul[:,1:] = np.cumsum(prob, axis = 1)
    for i in range(bs):
        new = enc.write(Y_raw[ind[i]], cumul[i,:].tolist())
    ind = ind + 1
    low_final = new[0] #fina low value 
    s_final = new[1]  #count of renormalizations
    bitstream = format(int(low_final), 'b')  
while len(bitstream) < (s_final+bitprecision):
    bitstream = '0' + bitstream
    print("encoded bit length:", len(bitstream))


    #START DECODER
    dec = ArithmeticDecoder(bitprecision, bitstream)

    symdec = []
    for symbols in range(len(sym)):
        symdec.append(dec.decode())
    print("decoded num stream:", symdec)

    #convert numbers back to letters
    dec_stream = []
    for x, sym in enumerate(symdec): 
        if sym == 0:
            dec_stream.append('a')
        elif sym ==1:
            dec_stream.append('c')
        elif sym == 2:
            dec_stream.append('g')
        elif sym == 3:
            dec_stream.append('t')

    dec_stream = dec_stream[:-1] 
    print("decoded symbol stream:", dec_stream)

NameError: name 'timesteps' is not defined

In [17]:
cumul

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0,

In [9]:

print(X)
print(X[ind,:].shape)

[[0 3 3 ... 3 3 1]
 [3 3 1 ... 3 1 2]
 [3 1 2 ... 1 2 2]
 ...
 [3 0 0 ... 1 1 1]
 [0 0 3 ... 1 1 3]
 [0 3 0 ... 1 3 1]]
(128, 64)


In [34]:
cumul = np.zeros((bs, alphabet_size+1), dtype = np.uint64)  
print(cumul[0,:].tolist())


[0, 0, 0, 0, 0]


In [11]:
print(sum(prob[1,:]))

1.0000000596046448


In [13]:
import arithmetic_coder

symbol stream input: ['a' 'c' 'g' 't' 't' 'a' 'c' 't' 'g' 'a' 't' 'a' 'a' 't' 'a' 'c' 'c' 'g'
 't']
num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3]
probability: [0.5 0.2 0.2 0.1]
cumulative prob: [0, 0.5, 0.7, 0.8999999999999999, 0.9999999999999999]
encoded bit length: 106
decoded num stream: [0, 1, 2, 3, 3, 0, 1, 3, 2, 0, 3, 0, 0, 3, 0, 1, 1, 2, 3, 1]
decoded symbol stream: ['a', 'c', 'g', 't', 't', 'a', 'c', 't', 'g', 'a', 't', 'a', 'a', 't', 'a', 'c', 'c', 'g', 't']


In [50]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = float)
cumul[1:] = np.cumsum(prob)        

In [51]:
cumul

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [8]:
prob = np.ones(alphabet_size)/alphabet_size
cumul = np.zeros(alphabet_size+1, dtype = np.uint64)
cumul[1:] = np.cumsum(prob*10000000 + 1)   

In [9]:
cumul

array([       0,  2500001,  5000002,  7500003, 10000004], dtype=uint64)